# **IMPLEMENTING A* ALGORITHM**

## **NAME: UMANG KIRIT LODAYA**
## **SAP ID: 60009200032**
## **BATCH: K - K1**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**IMPORTING LIBRARIES**

In [2]:
import pandas as pd
import math

**SETTING START AND END LOCATIONS**

In [3]:
START = {'id':0, 'name': 'Juhu Beach', 'lat': 19.10738206930842, 'long': 72.82651497989694} 
END = {'id':51, 'name': 'Gateway Of India', 'lat': 18.92205953640157, 'long': 72.8346486691006}

**IMPORTING LOCATIONS DATA**

In [4]:
df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/AI/locations.json')
df.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Ji..."
1,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Ju..."
2,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Ha..."
3,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Ma..."
4,FeatureCollection,"{'type': 'Feature', 'properties': {'name': 'Wi..."


In [5]:
df.shape

(50, 2)

**THUS, WE HAVE TAKEN 50 LOCATION WHICH MAY/MAY NOT BE IN BEST ROUTE**

#### **GENERATING THE LOCATIONS WITH LATITUDE AND LONGITUDE**

In [6]:
print(df['features'][0]['properties']['name'])
print(df['features'][0]['properties']['lat'], df['features'][0]['properties']['lon'])

Jio World Drive
19.0538461 72.85110462955777


In [7]:
data = {
    'name':[START['name']],
    'lat':[START['lat']],
    'long':[START['long']]
}

In [8]:
features = df["features"]
locations = []

for id, i in enumerate(features):
    new_location_props = i["properties"]
    new_location = {'id': id+1, "name": new_location_props["name"],
                    "lat": new_location_props["lat"],
                    "long": new_location_props["lon"],
                    "g": 0,
                    "h": 0,
                    "f": 0
                    }
                    
    data['name'].append(new_location['name'])
    data['lat'].append(new_location['lat'])
    data['long'].append(new_location['long'])
    
    locations.append(new_location)

locations.append(END)

In [9]:
data['name'].append(END['name'])
data['lat'].append(END['lat'])
data['long'].append(END['long'])

In [10]:
data = pd.DataFrame(data, index = list(range(len(data['name']))))
data.head()

,name,lat,long
0,Juhu Beach,19.107382,72.826515
1,Jio World Drive,19.053846,72.851105
2,Just Wear Dressline,19.066981,72.904310
3,Happiness Deli,18.961186,72.813282
4,Madcaps the Party Shop,19.057823,72.829794


In [11]:
data.tail()

,name,lat,long
47,Strand Book Stall,18.934187,72.834810
48,Maa Laxmi Jewelry,19.039256,72.852191
49,Star Auto Garage (Yusufbhai),19.045430,72.915750
50,ROOPAM PHARMACY,19.054337,72.891566
51,Gateway Of India,18.922060,72.834649


#### **FUNCTION**

**TO CHECK IF ITEM IS IN THE ARRAY**

In [12]:
def IN(item, array):
    for i in array:
        if item["id"]==i["id"]:
            return True
            
    return False

**FOR FILTERING THE OPEN/CLOSED ARRAY FOR REMOVING THE SEEN LOCATION**

In [13]:
def filterArray(item, array):
    
    filtered = list(filter(lambda x: x["id"]==item["id"], array))

    return filtered[0] if len(filtered)>0 else []

**FOR HAVERSINE DISTANCE**

In [14]:
def havesine_distance(loc1, loc2):
    lat1, long1 = loc1['lat'], loc1['long']
    lat2, long2 = loc2['lat'], loc2['long']

    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (long2 - long1) * math.pi / 180.0
 
    # convert to radians
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0
 
    # apply formulae
    a = (pow(math.sin(dLat / 2), 2) +
         pow(math.sin(dLon / 2), 2) *
             math.cos(lat1) * math.cos(lat2));
    rad = 6371
    c = 2 * math.asin(math.sqrt(a))
    return rad * c

**MOVE GEN FUNCTION**

In [15]:
def move_gen(location, open, closed):
    neighbours = []

    for i in locations:
        # FOR IMPLEMENTING REMOVE SEEN
        if not IN(i,open) and not IN(i, closed) and i["id"]!=location["id"]:
            neighbours.append(i)

    # RETURNING TOP 5 BEST NEXT LOCATIONS
    return sorted(neighbours, key=lambda x: havesine_distance(x, location))[:5]

**GOAL TEST FUNCTION**

In [16]:
def goal_test(location):
    global END
    if location["name"]==END["name"]:
        return True
        
    else:
        return False

**H(x) - HEURISTIC FUNCTION**

In [17]:
def h_function(location):
    global END
    h = havesine_distance(location, END)
    location["h"] = h

**G(x) - DISTANCE FROM START TO CURRENT LOCATION**

In [18]:
def g_function(parent, child):
    g = parent['g'] + havesine_distance(parent, child)
    child["g"] = g

**F(x) = G(x) + H(x)**

In [19]:
def f_function(item):
    item["f"] = item["g"] + item["h"]

**RECONSTRUCTING THE PATH**

In [20]:
def reconstructPath(my_path):
    goalPath = []

    # TAKING GOAL NODE AND ADDING IT TO PATH
    currNode = my_path.pop()
    goalPath.append(currNode[0])
    my_path = sorted(my_path, key=lambda x: x[0]["f"])

    # LOOP UNTIL WE GET ROOT NODE
    while currNode[1] != None:
        prev = []
        # FINDING PARENT NODE OF PARENT NODE OF CURRENT NODE
        for node in my_path:
            if node[0] == currNode[1]:
                prev.append(node)

        # print(f'PARENT OF {currNode[1]} ARE:',prev)
        # print('MY PATH:',my_path)
        
        # SETTING CURRENT NODE TO NEWLY FOUND PARENT NODE
        currNode = prev.pop()
        # ADDING NODE TO PATH
        goalPath.append(currNode[0])

    return goalPath[::-1]

**FOR DISPLAYING THE PATH**

In [21]:
def display(path):
    print('\nOPTIMAL PATH IS:\n')
    print(path[0], end=' ')
    for i in path[1:]:
        print('->', i, end=' ')

**A* ALGORITHM**

In [22]:
def A_STAR():
    global START
    START["g"] = 0
    
    h_function(START)
    START["f"] = START['g'] + START['h']

    OPEN = []
    OPEN.append(START)
    
    CLOSED = []
    SEEN = [[START, None]]

    while OPEN != []:
        # SORTING THE OPEN IN INCREASING OF F VALUE
        OPEN = sorted(OPEN, key=lambda x: x["f"])
        bestLocation = OPEN.pop(0)

        # GENERATING THE NEXT BEST LOCATIONS
        children = move_gen(bestLocation, OPEN, CLOSED)
        # print(len(children))
        
        for child in children:
            SEEN.append([child, bestLocation])

            if goal_test(child):
                CLOSED.append(bestLocation)
                CLOSED = reconstructPath(SEEN)
                return [i["name"] for i in CLOSED]
                
            else:
                g_function(bestLocation, child)
                h_function(child)
                f_function(child)
            
            
            if IN(child, OPEN):

                if filterArray(child, OPEN)["f"] < child["f"]:

                    # 3 - IF OPEN IN CLOSED AND DOESN'T HAVE BETTER F VALUE, THEN APPENDING TO THE OPEN
                    if IN(child, CLOSED) and filterArray(child, CLOSED)["f"] > child["f"]:

                            OPEN.append(child)
                
                # 2 - IF CHILD IN OPEN DOESN'T HAVE BETTER F VALUE
                else:
                    OPEN.append(child)
            
            # 1 - IF CHILD NOT IN OPEN, APPENDING IT
            else:
                OPEN.append(child)

        CLOSED.append(bestLocation)

In [23]:
path = A_STAR()
display(path)


OPTIMAL PATH IS:

Juhu Beach -> Avon Hair Dressers -> Patel Provision Stores -> S.K Creation -> Gateway Of India 